# Script to Preprocess Strain Reconstruction Test Data

- Using auxetic sensors (new with green wire)
- Initial length: 98 mm
- Date of data collection: 2023-01-17

> [1] Brett C. Hannigan, Tyler J. Cuthbert, Chakaveh Ahmadizadeh, and Carlo Menon. Distributed Sensing Along Fibres for Smart Clothing. *Science Advances*. 2023 (In Review).

©2023 ETH Zurich, Brett Hannigan; D-HEST; Biomedical and Mobile Health Technology (BMHT) Lab; Carlo Menon

## Import LCR Data
Files are named according to the format: `<sensor>_<strain>_<is_relaxation>`, where:
- `sensor` is one of A, B, C, D (from closest to readout port to furthers from readout port)
- `strain` is taken at levels of 0%, 10%, 20%, 30%, 40\%
- `is_relaxation` is FALSE for extension, TRUE for relaxation in preceeding step

In [1]:
RCparallel <- function(i, q, f) {
    i2q2 <- i^2+ q^2
    rp <- i2q2/i
    cp <- -1*q/(2*pi*f*i2q2)
    return(list(rp, cp))
}
RCseries <- function(i, q, f) {
    rs <- i
    cs <- -1/(2*pi*f*q)
    return(list(rs, cs))
}

In [4]:
avg_limit_low <- 100
avg_limit_high <- 400
PATH_PREFIX <- "./Synchronized"
REGEX_PATTERN <- "([A-D]{1,2})_Log_[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{6}_Test[0-9]{1,2}.tsv$"
files <- list.files(path=PATH_PREFIX, pattern=REGEX_PATTERN, no..=TRUE, recursive=TRUE)
f <- c(99609.0, 49805.0, 24902.0, 12451.0)
sensor_board <- data.frame()
for (file in files) {
    print(file)
    d <- read.csv(file = file.path(PATH_PREFIX, file), sep = '\t', header=TRUE, col.names=c("t", "position", "force", "displacement", "strain", "i0", "q0", "i1", "q1", "i2", "q2", "i3", "q3"))
    parsed_filename <- regexec(REGEX_PATTERN, file)
    start <- unlist(parsed_filename)[2]
    end <- start + attr(parsed_filename[[1]], "match.length")[2] - 1
    trial <- strtoi(substr(file, 4, 4))
    d$i0 <- d$i0*1.4253 -  82.30
    d$i1 <- d$i1*1.1516 - 125.95
    d$i2 <- d$i2*1.0519 - 108.39
    d$i3 <- d$i3*1.0344 - 110.18
    d$q0[d$i0<d$q0] <- d$q0[d$i0<d$q0]*6.1726 - 39098.0
    d$q0[d$i0>=d$q0] <- d$q0[d$i0>=d$q0]*2.0730 - 1068.5
    d$q1 <- d$q1*1.8172 - 2094.8
    d$q2 <- d$q2*1.2834 - 3640.2
    d$q3 <- d$q3*1.1780 - 1820.6
    d$i0 <- d$i0 - mean(d$i0[avg_limit_low:avg_limit_high])
    d$i1 <- d$i1 - mean(d$i1[avg_limit_low:avg_limit_high])
    d$i2 <- d$i2 - mean(d$i2[avg_limit_low:avg_limit_high])
    d$i3 <- d$i3 - mean(d$i3[avg_limit_low:avg_limit_high])
    d$q0 <- d$q0 - mean(d$q0[avg_limit_low:avg_limit_high])
    d$q1 <- d$q1 - mean(d$q1[avg_limit_low:avg_limit_high])
    d$q2 <- d$q2 - mean(d$q2[avg_limit_low:avg_limit_high])
    d$q3 <- d$q3 - mean(d$q3[avg_limit_low:avg_limit_high])
    I <- c(d$i0, d$i1, d$i2, d$i3)
    Q <- c(d$q0, d$q1, d$q2, d$q3)
    Zmag <- 2.0*sqrt(I^2 + Q^2)
    Zphase <- atan2(Q, I)*180/pi
    rpcp <- RCparallel(I, Q, rep(f, each=nrow(d)))
    # print(head(rpcp))
    # print(nrow(rpcp[[1]]))
    # print(nrow(Zmag))
    df <- data.frame(sensor=rep(substr(file, start[1], end[1]), length(f)*nrow(d)), 
                 trial=rep(trial, length(f)),
                 time=rep(d$t, length(f)),
                 strain=rep(d$strain, length(f)),
                 f=rep(f, each=nrow(d)),
                 I=I,
                 Q=Q,
                 Zmag=Zmag,
                 Zphase=Zphase,
                 Rp=rpcp[[1]],
                 Cp=rpcp[[2]])
    sensor_board <- rbind(sensor_board, df)
}
head(sensor_board)
save(sensor_board, file=file.path(PATH_PREFIX, "Data.Rda"))

[1] "Run0/A_Log_2023-01-21T154347_Test3.tsv"
[1] "Run0/AB_Log_2023-01-21T160242_Test9.tsv"
[1] "Run0/B_Log_2023-01-21T161940_Test15.tsv"
[1] "Run0/BC_Log_2023-01-21T163724_Test21.tsv"
[1] "Run0/C_Log_2023-01-21T165419_Test27.tsv"
[1] "Run0/CD_Log_2023-01-21T171153_Test33.tsv"
[1] "Run0/D_Log_2023-01-21T174722_Test45.tsv"
[1] "Run1/A_Log_2023-01-21T154746_Test4.tsv"
[1] "Run1/AB_Log_2023-01-21T160532_Test10.tsv"
[1] "Run1/B_Log_2023-01-21T162247_Test16.tsv"
[1] "Run1/BC_Log_2023-01-21T164005_Test22.tsv"
[1] "Run1/C_Log_2023-01-21T165704_Test27.tsv"
[1] "Run1/CD_Log_2023-01-21T171455_Test34.tsv"
[1] "Run1/D_Log_2023-01-21T175140_Test46.tsv"
[1] "Run2/A_Log_2023-01-21T155059_Test5.tsv"
[1] "Run2/AB_Log_2023-01-21T160825_Test11.tsv"
[1] "Run2/B_Log_2023-01-21T162540_Test17.tsv"
[1] "Run2/BC_Log_2023-01-21T164237_Test23.tsv"
[1] "Run2/C_Log_2023-01-21T165953_Test29.tsv"
[1] "Run2/CD_Log_2023-01-21T171758_Test35.tsv"
[1] "Run2/D_Log_2023-01-21T175432_Test47.tsv"
[1] "Run3/A_Log_2023-01-21T15

,sensor,trial,time,strain,f,I,Q,Zmag,Zphase,Rp,Cp
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,A,0,0.00,0.000000e+00,99609,1.306422,-3.642874,7.740096,-70.27094,11.46434,3.886271e-07
2,A,0,0.01,2.170742e-06,99609,1.306422,-4.183927,8.766295,-72.65914,14.70580,3.479633e-07
3,A,0,0.02,4.222312e-06,99609,1.306422,-4.727053,9.808521,-74.55072,18.41041,3.140256e-07
4,A,0,0.03,6.851208e-06,99609,1.306422,-5.268106,10.855354,-76.07235,22.54989,2.857249e-07
5,A,0,0.04,1.054466e-05,99609,1.306422,-5.809159,11.908496,-77.32561,27.13752,2.618068e-07
6,A,0,0.05,1.421531e-05,99609,1.306422,-6.350212,12.966408,-78.37480,32.17331,2.413962e-07
